### Section 4.6: Make a pretty image

In [21]:
# Importing necessary packages 
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import astroalign as aa

In [22]:
def read_pretty_images(file_template, start_index, end_index):
    """
    Read the pretty images (red, blue, visible) of the Ring Nebula.
    
    :param file_template: Template path for the pretty images with placeholders for the frame number.
    :type file_template: str
    :param start_index: The starting index for the image frames.
    :type start_index: int
    :param end_index: The ending index for the image frames.
    :type end_index: int
    :return: A list containing the data and headers of the loaded FITS files.
    """
    # List to store the FITS data and headers
    pretty_images_info = []
    # Loop through each file and read the data
    for i in range(start_index, end_index + 1):
        # Construct the file name based on the frame number
        file_path = file_template.format(i)
        # Open the FITS file and read the header and data
        with fits.open(file_path) as hdul:
            header = hdul[0].header
            data = hdul[0].data
            # Store the header and data
            pretty_images_info.append((header, data))
    
    return pretty_images_info

In [23]:
def read_dark_frames(file_template, start_index, end_index):
    """
    Read dark frames and output their exposure times and temperatures.
    
    :param file_template: Template path for the dark files with placeholders for the frame number.
    :type file_template: str
    :param start_index: The starting index for the dark frames.
    :type start_index: int
    :param end_index: The ending index for the dark frames.
    :type end_index: int
    :return: A list containing the data and headers of the loaded dark frames.
    """
    # List to store the FITS data and headers
    dark_frame_info = []
    # Loop through each dark frame and read the data
    for i in range(start_index, end_index + 1):
        # Construct the file name based on the frame number
        file_path = file_template.format(i)
        # Open the FITS file and read the header and data
        with fits.open(file_path) as hdul:
            header = hdul[0].header
            data = hdul[0].data
            # Store the header and data
            dark_frame_info.append((header, data))
    
    return dark_frame_info

In [24]:
def calculate_median(data_list):
    """
    Calculate the median of a list of 2D arrays.
    
    :param data_list: List of 2D arrays (images).
    :type data_list: list
    :return: Median of the 2D arrays.
    """
    return np.median(np.array(data_list), axis=0)

In [25]:
def subtract_median_dark_from_pretty(median_dark, median_pretty):
    """
    Subtract the median dark frame from the median pretty image.
    
    :param median_dark: Median dark frame.
    :type median_dark: ndarray
    :param median_pretty: Median pretty image.
    :type median_pretty: ndarray
    :return: Subtracted image.
    """
    return median_pretty - median_dark

In [26]:
# Templates for the file names
dark_frame_file_template = r"..\FITS_Files\ring_nebula_dark_frame_10_sec.{:08d}.DARK.FIT"
red_file_template = r"..\FITS_Files\ring_nebula_for_10_sec_red.{:08d}.FIT"
blue_file_template = r"..\FITS_Files\ring_nebula_for_10_sec_blue.{:08d}.FIT"
visible_file_template = r"..\FITS_Files\ring_nebula_for_10_sec_visible.{:08d}.FIT"

# Define the starting and ending indices
dark_start_index = 4
dark_end_index = 6
pretty_start_index = 0
pretty_end_index = 2

# Read dark frames and pretty images
dark_frames = read_dark_frames(dark_frame_file_template, dark_start_index, dark_end_index)
pretty_red_images = read_pretty_images(red_file_template, pretty_start_index, pretty_end_index)
pretty_blue_images = read_pretty_images(blue_file_template, pretty_start_index, pretty_end_index)
pretty_visible_images = read_pretty_images(visible_file_template, pretty_start_index, pretty_end_index)

# Extract the image data from the dark frames and pretty images
dark_data = [data for header, data in dark_frames]
red_data = [data for header, data in pretty_red_images]
blue_data = [data for header, data in pretty_blue_images]
visible_data = [data for header, data in pretty_visible_images]

# Calculate medians
median_dark = calculate_median(dark_data)
median_red = calculate_median(red_data)
median_blue = calculate_median(blue_data)
median_visible = calculate_median(visible_data)

# Subtract the median dark frame from the median pretty images
subtracted_red = subtract_median_dark_from_pretty(median_dark, median_red)
subtracted_blue = subtract_median_dark_from_pretty(median_dark, median_blue)
subtracted_visible = subtract_median_dark_from_pretty(median_dark, median_visible)

In [28]:
# Align the pretty images to the green (visible) image
calibrated_images = [subtracted_visible, subtracted_blue, subtracted_red]

aligned_blue, footprint = aa.register(calibrated_images[0], calibrated_images[1])  
aligned_red, footprint = aa.register(calibrated_images[2], calibrated_images[1])  

# Save aligned images as FITS files
fits.PrimaryHDU(aligned_blue).writeto("aligned_blue.fits", overwrite=True)
fits.PrimaryHDU(aligned_red).writeto("aligned_red.fits", overwrite=True)
fits.PrimaryHDU(calibrated_images[0]).writeto("aligned_green.fits", overwrite=True)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'aligned_green.fits'